In [61]:
#Teamno1 
#Madhavan M
#Sriram V
#siddarth
#auston



#Data preprocessing is done as the ratio of no:yes is 5:1 and saved as cleaned dataset

import pandas as pd

# Load the dataset from CSV
df = pd.read_csv('/kaggle/input/datathon/DATATHON_EVENT_DATASET.csv')

# Drop the first 30,000 rows from the dataframe as it contains no
df_dropped = df.iloc[30000:].reset_index(drop=True)

# Display the shape of the updated dataframe
print("Shape of dataframe after dropping 30,000 rows:", df_dropped.shape)

# Save the cleaned dataframe to a new CSV file
df_dropped.to_csv('cleaned_dataset.csv', index=False)

print("Cleaned dataset saved successfully.")


Shape of dataframe after dropping 30,000 rows: (28150, 11)
Cleaned dataset saved successfully.


In [62]:
# Import libraries
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.impute import SimpleImputer
import numpy as np

# Load the dataset from cleaned data CSV
# This dataset has already undergone initial cleaning (like dropping irrelevant rows)
df = pd.read_csv('/kaggle/input/cleaned-dataset/cleaned_dataset.csv')

# ---- Data Cleaning (Handling missing values, outliers) ----
# Using SimpleImputer to handle missing values based on the type of data.
# Numerical columns are filled with the median (to avoid skewing the data with extreme values).
# Categorical columns are filled with the most frequent value (mode) to ensure consistency.
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Impute missing values
df[numerical_cols] = SimpleImputer(strategy='median').fit_transform(df[numerical_cols])
df[categorical_cols] = SimpleImputer(strategy='most_frequent').fit_transform(df[categorical_cols])

# ---- Outlier Handling (Optional but can be added here if needed) ----
# Outliers can be identified and removed or handled based on business rules or statistical methods such as IQR or Z-score.

# ---- Label Conversion ----
# Convert 'Yes'/'No' in 'Fraud' column to 1 and 0 for binary classification.
df['Fraud'] = df['Fraud'].map({'Yes': 1, 'No': 0})

# ---- Drop the 'Expected_Fraud' column ----
# The 'Expected_Fraud' column has no predictive value (contains only 'No') and thus is removed from the dataset.
df = df.drop(columns=['Expected_Fraud'])

# ---- Define Features (X) and Target (y) ----
# The target variable 'Fraud' is what we are trying to predict. It represents whether a transaction is fraudulent or not.
# All other columns are features (X) used as input to the model.
X = df.drop(columns=['Fraud'])  # Features
y = df['Fraud']  # Target (fraud label)

# ---- Handling High Cardinality Columns ----
# 'Origin_ID' and 'Destination_ID' have many unique values (high cardinality).
# Label encoding is used to convert these categorical values into numeric labels while preserving their uniqueness.
label_encoder = LabelEncoder()
X['Origin_ID'] = label_encoder.fit_transform(X['Origin_ID'])
X['Destination_ID'] = label_encoder.fit_transform(X['Destination_ID'])

# ---- One-Hot Encoding for Categorical Columns ----
# The 'Transaction_Type' column has a small number of unique values and is encoded using One-Hot Encoding.
# One-Hot Encoding creates binary columns for each category, making it easier for the model to understand categorical data.
categorical_cols = ['Transaction_Type']
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(drop='first'), categorical_cols)],  # One-Hot Encoding
    remainder='passthrough'  # Leave numerical columns unchanged
)

# ---- Create the Pipeline ----
# A pipeline is used to combine preprocessing steps and the model into a single workflow.
# The pipeline first applies One-Hot Encoding, then scales numerical data, and finally trains a Random Forest classifier.
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Preprocessing: One-Hot Encoding and pass-through other features
    ('scaler', StandardScaler()),  # Scale numerical features to have mean=0 and variance=1 for better model performance
    ('classifier', RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42))  # Random Forest Model
])

# ---- Splitting the Data ----
# The dataset is split into 60% training and 40% testing using stratification to maintain the balance between classes (fraud vs. non-fraud).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

# ---- Train the Model ----
# The Random Forest Classifier is trained on the training set. It is an ensemble model that reduces overfitting and increases accuracy.
pipeline.fit(X_train, y_train)

# ---- Predictions and Evaluation ----
# The trained model is used to make predictions on the test set.
y_pred = pipeline.predict(X_test)

# ---- Model Performance Evaluation ----
# F1 Score, Confusion Matrix, and Accuracy are used to evaluate the model.
# F1 Score is the harmonic mean of precision and recall, suitable for imbalanced datasets like fraud detection.
# The confusion matrix shows true/false positives and negatives, giving insight into the types of errors.
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

# Print the evaluation metrics
print("Our model")
print(f'Accuracy: {accuracy * 100:.2f}%')  # High accuracy indicates the model is making correct predictions most of the time.
print(f'F1-Score: {f1}')  # F1 score ensures a balance between false positives and false negatives.
print('Confusion Matrix:')
print(cm)  # Provides details on how many frauds were correctly identified and how many were missed.

# ---- Save the Trained Model ----
# The entire pipeline (including preprocessing and model) is saved using joblib for later use.
# This allows the model to be reloaded and used without needing retraining.
joblib.dump(pipeline, 'fraud_detection_pipeline.pkl')
print("Model saved successfully.")

#the output we got
#Accuracy: 98.21%
#F1-Score: 0.968967114404817
#Confusion Matrix:
#[[7921   79]
# [ 122 3138]]
#Model saved successfully.


Our model
Accuracy: 98.21%
F1-Score: 0.968967114404817
Confusion Matrix:
[[7921   79]
 [ 122 3138]]
Model saved successfully.


In [63]:
# Import libraries
#wholedataset
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.impute import SimpleImputer

# Load the dataset from cleaned data CSV
df = pd.read_csv('/kaggle/input/datathon/DATATHON_EVENT_DATASET.csv')

# ---- Data Cleaning (Handling missing values) ----
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Impute missing values
df[numerical_cols] = SimpleImputer(strategy='median').fit_transform(df[numerical_cols])
df[categorical_cols] = SimpleImputer(strategy='most_frequent').fit_transform(df[categorical_cols])

# ---- Label Conversion ----
df['Fraud'] = df['Fraud'].map({'Yes': 1, 'No': 0})

# ---- Drop the 'Expected_Fraud' column ----
df = df.drop(columns=['Expected_Fraud'])

# ---- Define Features (X) and Target (y) ----
X = df.drop(columns=['Fraud'])  # Features
y = df['Fraud']  # Target (fraud label)

# ---- Handling High Cardinality Columns ----
label_encoder = LabelEncoder()
X['Origin_ID'] = label_encoder.fit_transform(X['Origin_ID'])
X['Destination_ID'] = label_encoder.fit_transform(X['Destination_ID'])

# ---- One-Hot Encoding for Categorical Columns ----
categorical_cols = ['Transaction_Type']
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(drop='first'), categorical_cols)],
    remainder='passthrough'
)

# ---- Create the Pipeline ----
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42))
])

# ---- Train the Model ----
pipeline.fit(X, y)  # Train on the entire dataset

# ---- Predictions and Evaluation ----
y_pred = pipeline.predict(X)

# ---- Model Performance Evaluation ----
f1 = f1_score(y, y_pred)
cm = confusion_matrix(y, y_pred)
accuracy = accuracy_score(y, y_pred)

# Print the evaluation metrics
print("whole dataset")
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'F1-Score: {f1}')
print('Confusion Matrix:')
print(cm)

# ---- Save the Trained Model ----
joblib.dump(pipeline, 'fraud_detection_pipeline_full.pkl')
print("Model saved successfully.")

#Accuracy: 98.64%
#F1-Score: 0.9495272169690775
#Confusion Matrix:
#[[49929    71]
# [  719  7431]]
#Model saved successfully.

whole dataset
Accuracy: 98.64%
F1-Score: 0.9495272169690775
Confusion Matrix:
[[49929    71]
 [  719  7431]]
Model saved successfully.


In [64]:
# Import necessary libraries
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

# Load the cleaned test dataset
df_test = pd.read_csv('/kaggle/input/test-dataset/Test.csv')  # Replace with your test dataset path

# ---- Test Data Preprocessing ----
# Impute missing values for numerical and categorical columns
numerical_cols = df_test.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df_test.select_dtypes(include=['object']).columns

# Impute missing values (the same strategy used during training)
df_test[numerical_cols] = SimpleImputer(strategy='median').fit_transform(df_test[numerical_cols])
df_test[categorical_cols] = SimpleImputer(strategy='most_frequent').fit_transform(df_test[categorical_cols])

# Convert 'Yes'/'No' in 'Fraud' column to 1 and 0
df_test['Fraud'] = df_test['Fraud'].map({'Yes': 1, 'No': 0})

# Drop the 'Expected_Fraud' column if it exists
df_test = df_test.drop(columns=['Expected_Fraud'], errors='ignore')

# Define features and target for the test data
X_test = df_test.drop(columns=['Fraud'])  # Test features
y_test = df_test['Fraud']  # Test target (fraud label)

# ---- Label Encoding for High-Cardinality Columns ----
# Ensure that the same LabelEncoder is applied to the test data
# Load the LabelEncoder (apply the same transformation as done during training)
label_encoder = LabelEncoder()
X_test['Origin_ID'] = label_encoder.fit_transform(X_test['Origin_ID'])
X_test['Destination_ID'] = label_encoder.fit_transform(X_test['Destination_ID'])

# Load the saved model pipeline
model = joblib.load('fraud_detection_pipeline.pkl')

# ---- Test the Model ----
# Make predictions using the loaded model
y_pred = model.predict(X_test)

# ---- Evaluate the Model ----
# Calculate F1 Score, Confusion Matrix, and Accuracy
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

# Print the evaluation metrics
print("Test results")
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'F1-Score: {f1}')
print('Confusion Matrix:')
print(cm)

#Accuracy: 99.45%
#F1-Score: 0.9841269841269841
#Confusion Matrix:
#[[302   1]
# [  1  62]]

Test results
Accuracy: 99.45%
F1-Score: 0.9841269841269841
Confusion Matrix:
[[302   1]
 [  1  62]]
